# 지난 시간 복습
1. 가변 길이 인수(step11의 개선 편)
2. 가변 길이 인수(역전파)
3. 같은 변수 반복 사용

In [13]:
import numpy as np



In [12]:
outer()
outer()

[1]
[1]


# 이번주에는 총 4가지를 배운다
1. 복잡한 계산 그래프(이론)
2. 복잡한 계산 그래프(구현)
3. 메모리 관리와 순환 참조
4. 메모리 절약 모드

1,2 는 **_유향그래프의 위상 정렬_** 에 관한 내용  
3,4 는 파이썬 언어의 **_GC_** 에 관해서, 그리고 **_메모리 개선_** 을 구현  

# Not Clone Coding

In [14]:
import weakref

def as_array(x):
    if not isinstance(x, np.ndarray):
        return np.array(x)

class Variable:
    def __init__(self, data):
        self.data = data
        self.creator = None
        self.grad = None
        self.generation = None

    def set_creator(self, func):
        self.creator = func
        self.generation = self.creator.generation + 1

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) #### ambiguous

        seen_func = {self.creator}
        funcs = [self.creator]                  #### 힙으로 바꾸면 좋음

        while funcs:
            func = funcs.pop()
            inputs = func.inputs

            gys = [output.grad for output in func.outputs]
            gxs = func.backward(*gys)
            if isinstance(gxs, tuple):
                gxs = (gxs, )

            for input, gx in zip(inputs, gxs):
                input.grad = gx
                if input.creator not in seen_func:
                    funcs.append(input.creator)
                    seen_func.add(input.creator)
            funcs.sort(key=lambda x: -x.generation)

class Function:
    def __call__(self, inputs):
        if self.retain_grad == True:
            self.inputs = inputs
        inputs = [input.data for input in self.inputs]
        outputs = [as_array(output) for output in self.forward(*inputs)]
        outputs = list(map(Variable, outputs))
        self.outputs = outputs

        self.generation = max([input.generation for input in self.inputs])

    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()

class Add(Function):
    def forward(self, x0, x1):
        return x0 + x1
    
    def backward(self, gy):
        return gy, gy

# 1. 복잡한 계산(이론)

